In [1]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import pickle

import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

Left_Line = Line()
Right_Line = Line()

In [3]:
def undistort(img):
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

def hls_transform(img, thresh):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    sat = hls[:,:,2]
    binary = np.zeros_like(sat)
    binary[(sat > thresh[0]) & (sat <= thresh[1])] = 1
    return binary

def abs_grad(img, sobel_kernel, thresh, orient):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, sobel_kernel))
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary = np.zeros_like(scaled_sobel)
    binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return binary

def mag_grad(img, sobel_kernel, thresh):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    binary = np.zeros_like(gradmag)
    binary[(gradmag >= thresh[0]) & (gradmag <= thresh[1])] = 1
    return binary

def dir_grad(img, sobel_kernel, thresh):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    graddir = np.arctan2(np.absolute(sobely),np.absolute(sobelx))
    binary = np.zeros_like(graddir)
    binary[(graddir >= thresh[0]) & (graddir <= thresh[1])] = 1
    return binary

def combined_gradient(img, ksize):
    gradx = abs_grad(img, sobel_kernel=ksize, thresh=(100, 255), orient='x')
    grady = abs_grad(img, sobel_kernel=ksize, thresh=(100, 255), orient='y')
    mag_binary = mag_grad(img, sobel_kernel=ksize, thresh=(100, 255))
    dir_binary = dir_grad(img, sobel_kernel=ksize, thresh=(np.pi/4, np.pi/2))
    
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    return combined

def pers_trans(img, src, dst):
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size)
    return warped

def histo(img):
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    return histogram

In [4]:
camera_calibration_values = open('cal_coeff.p', 'rb') 
camera_calibration_values = pickle.load(camera_calibration_values)
mtx = camera_calibration_values['mtx'] 
dist = camera_calibration_values['dist']

EOFError: Ran out of input

In [4]:
def lane_finder(img):
    #[mtx, dist] = pickle.load( open( "cal_coeff.p", "rb" ) )
    
    undist = img
    
    hls = hls_transform(undist, (100,255))
    grad = combined_gradient(undist, 3)
    
    mixed = np.zeros_like(grad)
    mixed[(grad == 1) | (hls == 1)] = 1
    
    img_size = (undist.shape[1], undist.shape[0])

    src = np.float32([[np.int(img_size[0]*0.435), np.int(img_size[1]*0.65)],
                      [np.int(img_size[0]*0.565), np.int(img_size[1]*0.65)],
                      [np.int(img_size[0]*0.9), img_size[1]],
                      [np.int(img_size[0]*0.1), img_size[1]]])
    offset = 150

    dst = np.float32([[offset, 0], 
                      [img_size[0]-offset, offset], 
                      [img_size[0]-offset, img_size[1]], 
                      [offset, img_size[1]]])
    
    warped = pers_trans(mixed,src,dst)
    histogram = histo(warped)
    
    
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((warped, warped, warped))*255

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(warped.shape[0]/nwindows)

    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = warped.shape[0] - (window+1)*window_height
        win_y_high = warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 10) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 10) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    Right_Line.allxrightx = nonzerox[right_lane_inds]
    Right_Line.allyrighty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    Left_line.current_fitleft_fit = np.polyfit(lefty, leftx, 2)
    Right_Line.current_fitright_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
    Left_Line.allxleft_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    Riright_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    # Define conversions in x and y from pixels space to meters
    y_eval = np.max(ploty)
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    #position from center
    lane_center = 640
    right_distance = right_fitx[-1]-lane_center
    left_distance = lane_center - left_fitx[-1]
    pos_from_center = np.absolute(right_distance - left_distance)*xm_per_pix
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    print(left_curverad, 'm', right_curverad, 'm')
    # Example values: 632.1 m    626.2 m
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    fitted_lane_image = np.zeros_like(warped)
    # Draw the lane onto the warped blank image
    cv2.fillPoly(fitted_lane_image, np.int_([pts]), 1)

    unwarp = pers_trans(fitted_lane_image, dst, src)
    lanes=np.zeros_like(undist)
    lanes[(unwarp>0)]=(0,255,0)
    
    #cv2.putText(undis_im, [np.int(im_feat.left_curverad), np.int(im_feat.right_curverad)], (np.int(undis_im.shape[1]*0.6),np.int(undis_im.shape[0]*0.3)), cv2.FONT_HERSHEY_SIMPLEX, 6,(0,0,0),13)
    
    overlayed = cv2.addWeighted(undist, 1, lanes, 0.3, 0)
    
    left_text = 'Left lane curvature: %dm' %left_curverad
    right_text = 'Right lane curvature: %dm' %right_curverad
    pos_text = 'Position from lane center: %.3fm'%pos_from_center
    cv2.putText(overlayed, left_text, (np.int(overlayed.shape[1]*0.55),np.int(overlayed.shape[0]*0.05)), font, 1, (0,255,0), 4, cv2.LINE_AA)
    cv2.putText(overlayed, right_text, (np.int(overlayed.shape[1]*0.55),np.int(overlayed.shape[0]*0.1)), font, 1, (0,255,0), 4, cv2.LINE_AA)
    cv2.putText(overlayed, pos_text, (np.int(overlayed.shape[1]*0.55),np.int(overlayed.shape[0]*0.15)), font, 1, (0,255,0), 4, cv2.LINE_AA)
    
    return overlayed
    

In [5]:
output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
road_clip = clip1.fl_image(lane_finder) #NOTE: this function expects color images!!
%time road_clip.write_videofile(output, audio=False)

679.682201769 m 1012.49841682 m
(4, 2)
(720, 1280)
(720, 1280, 3)
[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


  0%|                                                 | 0/1261 [00:00<?, ?it/s]

679.682201769 m 1012.49841682 m
(4, 2)
(720, 1280)
(720, 1280, 3)


  0%|                                         | 1/1261 [00:00<04:47,  4.39it/s]

689.403967446 m 1276.3063053 m
(4, 2)
(720, 1280)
(720, 1280, 3)


...

5589.12824573 m 4594.92293387 m
(4, 2)
(720, 1280)
(720, 1280, 3)


100%|█████████████████████████████████████▉| 1259/1261 [04:38<00:00,  4.30it/s]

6241.93541183 m 2160.4445946 m
(4, 2)
(720, 1280)
(720, 1280, 3)


100%|█████████████████████████████████████▉| 1260/1261 [04:39<00:00,  4.28it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

Wall time: 4min 39s


In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

In [7]:
challenge_output = 'challenge_video_output.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
road_clip = clip1.fl_image(lane_finder) #NOTE: this function expects color images!!
%time road_clip.write_videofile(challenge_output, audio=False)

255.060790042 m 403.809538098 m
(4, 2)
(720, 1280)
(720, 1280, 3)
[MoviePy] >>>> Building video challenge_video_output.mp4
[MoviePy] Writing video challenge_video_output.mp4


  0%|                                                  | 0/485 [00:00<?, ?it/s]

255.060790042 m 403.809538098 m
(4, 2)
(720, 1280)
(720, 1280, 3)


  0%|                                          | 1/485 [00:00<01:38,  4.90it/s]

241.719456494 m 242.544644604 m
(4, 2)
(720, 1280)
(720, 1280, 3)


...

747.374034043 m 3.61257249838 m
(4, 2)
(720, 1280)
(720, 1280, 3)


100%|███████████████████████████████████████▉| 484/485 [01:38<00:00,  4.97it/s]

747.374034043 m 3.61257249838 m
(4, 2)
(720, 1280)
(720, 1280, 3)


100%|████████████████████████████████████████| 485/485 [01:38<00:00,  4.95it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_video_output.mp4 

Wall time: 1min 39s


In [8]:
harder_challenge_output = 'harder_challenge_video_output.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4")
road_clip = clip1.fl_image(lane_finder) #NOTE: this function expects color images!!
%time road_clip.write_videofile(harder_challenge_output, audio=False)

397.74444447 m 485.000339135 m
(4, 2)
(720, 1280)
(720, 1280, 3)
[MoviePy] >>>> Building video harder_challenge_video_output.mp4
[MoviePy] Writing video harder_challenge_video_output.mp4


  0%|                                                 | 0/1200 [00:00<?, ?it/s]

397.74444447 m 485.000339135 m
(4, 2)
(720, 1280)
(720, 1280, 3)


  0%|                                         | 1/1200 [00:00<04:53,  4.08it/s]

418.028773088 m 303.129588069 m
(4, 2)
(720, 1280)
(720, 1280, 3)


...

1166.524123 m 52.8314599658 m
(4, 2)
(720, 1280)
(720, 1280, 3)


 84%|███████████████████████████████▊      | 1006/1200 [03:54<00:40,  4.75it/s]

210.173553248 m 207.619134166 m
(4, 2)
(720, 1280)
(720, 1280, 3)


 84%|███████████████████████████████▉      | 1007/1200 [03:54<00:40,  4.74it/s]

TypeError: expected non-empty vector for x